# Machine Translation Project
A deep neural network that functions as part of an end-to-end machine translation pipeline

The completed pipeline should accept English text as input and return the french translation as output. 
The big picture of this project will go in this order :) 

- **Preprocess** - It'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, One can engage in their own investigations, to design their own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections 

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam 
from keras.losses import sparse_categorical_crossentropy

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Verifying access to the GPU
The following test applies only if one expects to be using a GPU, e.g., while running in a School Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3427585778395069873
]


## Dataset 
This dataset will be used to train and evaluate the pipeline. This dataset contains a small vocabulary. This dataset will allow the model to be trained in a reasonable amount of time. 
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Loading english data
english_sentences = helper.load_data('data/small_vocab_en')
# Loading french data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')
print(type(english_sentences))


Dataset Loaded
<class 'list'>


## Files 
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`. View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line{}: {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line{}: {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line1: new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line1: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line2: the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line2: les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, One can see they have been preprocessed already. The puncuations have been delimited using spaces. All the text have been converted to lowercase. This should save some time, but the text requires more preprocessing.
### Vocabulary 
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset it'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, It won't use text data as input to the model. Instead, it'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (1750)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

One can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so it'll use those.

Below is a dsiplay of executing each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. One can use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: 
    
    # Creates the tokeninzer
    t = Tokenizer()
    # Creating dictionary mapping words (str) to their rank/index (int)
    t.fit_on_texts(x)
    # Using the tokenizer to tokenize the text
    text_sequences = t.texts_to_sequences(x)
    return text_sequences, t
tests.test_tokenize(tokenize)


# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding(1750)
When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, One can add padding to the end of the sequences to make them the same length.

It's important to build this in a way that all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: 
    
    # If length equals None, set it to be the length of the longest sequence in x
    if length == None:
        length = len(max(x, key=len))
        
    # Using Keras's pad_sequences to pad the sequences with 0's
    padded_sequences = pad_sequences(sequences=x, maxlen=length, padding='post', value=0)
    
    return padded_sequences

tests.test_pad(pad)

# Padding Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
The goal for this project is to build neural network architecture, Let's create a preprocess pipeline and provide the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


### Extension - Splitting the data
Using `sklearn.model_selection.train_test_split` to split the data into training and test sets.

Note: `train_test_split` should be perferred on X and y data together, not seperately.

For example,

`train_test_split(X, y, test_size = 0.2, random_state = 42)`

Do not split the data seperately (as seen as the commented out code in the below cell).

In [10]:
### Splits the unprocessed data into training and test sets using Sklearn ###

english_sentences_train, english_sentences_test, french_sentences_train, french_sentences_test =\
    train_test_split(english_sentences, french_sentences, test_size=0.2, random_state=42)
    
    
### Failed attempts below ###

#french_sentences_train, french_sentences_test = train_test_split(french_sentences,
                                                                 #test_size=0.2)
    
#english_sentences_train, english_sentences_test = train_test_split(english_sentences,
                                                                 #test_size=0.2)

In [11]:
# Calculating the training set stats

english_train_words_counter = collections.Counter([word for sentence in english_sentences_train for word in sentence.split()])
french_train_words_counter = collections.Counter([word for sentence in french_sentences_train for word in sentence.split()])

print('{} English words in the training set.'.format(len([word for sentence in english_sentences_train for word in sentence.split()])))
print('{} unique English words in the training set.'.format(len(english_train_words_counter)))
print('10 Most common words in the English training dataset:')
print('"' + '" "'.join(list(zip(*english_train_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words in the training set.'.format(len([word for sentence in french_sentences_train for word in sentence.split()])))
print('{} unique French words in the training set.'.format(len(french_train_words_counter)))
print('10 Most common words in the French training dataset:')
print('"' + '" "'.join(list(zip(*french_train_words_counter.most_common(10)))[0]) + '"')

1458806 English words in the training set.
227 unique English words in the training set.
10 Most common words in the English training dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1568964 French words in the training set.
354 unique French words in the training set.
10 Most common words in the French training dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [12]:
#2 Calculating the test set stats
english_test_words_counter = collections.Counter([word for sentence in english_sentences_test for word in sentence.split()])
french_test_words_counter = collections.Counter([word for sentence in french_sentences_test for word in sentence.split()])

print('{} English words in the test set.'.format(len([word for sentence in english_sentences_test for word in sentence.split()])))
print('{} unique English words in the test set.'.format(len(english_test_words_counter)))
print('10 Most common words in the English test dataset:')
print('"' + '" "'.join(list(zip(*english_test_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words in the test set.'.format(len([word for sentence in french_sentences_test for word in sentence.split()])))
print('{} unique French words in the test set.'.format(len(french_test_words_counter)))
print('10 Most common words in the French test dataset:')
print('"' + '" "'.join(list(zip(*french_test_words_counter.most_common(10)))[0]) + '"')

364444 English words in the test set.
227 unique English words in the test set.
10 Most common words in the English test dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

392331 French words in the test set.
338 unique French words in the test set.
10 Most common words in the French test dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


### Preprocessing the training data

Using `preprocess` to process the training split of the data. 

In [13]:
# Preprocessing the training split of the data

preproc_english_sentences_train, preproc_french_sentences_train, english_tokenizer_train, french_tokenizer_train =\
    preprocess(english_sentences_train, french_sentences_train)
    
max_english_sequence_length_train = preproc_english_sentences_train.shape[1]
max_french_sequence_length_train = preproc_french_sentences_train.shape[1]
english_vocab_size_train = len(english_tokenizer_train.word_index)
french_vocab_size_train = len(french_tokenizer_train.word_index)

print('Training Data Preprocessed')
print("Max English train dataset sentence length:", max_english_sequence_length_train)
print("Max French train dataset sentence length:", max_french_sequence_length_train)
print("English train dataset vocabulary size:", english_vocab_size_train)
print("French train dataset vocabulary size:", french_vocab_size_train)

Training Data Preprocessed
Max English train dataset sentence length: 15
Max French train dataset sentence length: 21
English train dataset vocabulary size: 199
French train dataset vocabulary size: 344


### Preprocessing the training data

Using `preprocess` to process the test split of the data. 

In [14]:
# Preprocessing the test split of the data 

preproc_english_sentences_test, preproc_french_sentences_test, english_tokenizer_test, french_tokenizer_test =\
    preprocess(english_sentences_test, french_sentences_test)
    
max_english_sequence_length_test = preproc_english_sentences_test.shape[1]
max_french_sequence_length_test = preproc_french_sentences_test.shape[1]
english_vocab_size_test = len(english_tokenizer_test.word_index)
french_vocab_size_test = len(french_tokenizer_test.word_index)

print('Test Data Preprocessed')
print('Max English test dataset sentence length:', max_english_sequence_length_test)
print('Max French test dataset sentence length:', max_french_sequence_length_test)
print('English test datset vocab size:', english_vocab_size_test)
print('French test dataset vocab size', french_vocab_size_test)
print(preproc_french_sentences_test.shape)

Test Data Preprocessed
Max English test dataset sentence length: 15
Max French test dataset sentence length: 20
English test datset vocab size: 199
French test dataset vocab size 327
(27573, 20, 1)


## Models
In this section, The experiment connects various neural network architectures.
It begins by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, It constructs a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form that is wanted. The goal is to want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation. It'll be using this function to better understand the output of the neural network.

In [15]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (1750)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, It'll build a RNN that translates English to French.

In [17]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Building the layers
    learning_rate=0.001
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate))
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Training the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=4096, epochs=10, validation_split=0.2)

# Printing prediction(s).
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
106496/110288 [===========================>..] - ETA: 4s - loss: 5.3361 

InvalidArgumentError: Received a label value of 345 which is outside the valid range of [0, 345).  Label values: 62 1 18 37 11 44 5 3 1 12 21 15 42 0 0 0 0 0 0 0 0 36 1 12 68 2 50 6 3 1 67 2 49 0 0 0 0 0 0 0 0 0 36 1 8 19 2 50 6 3 1 9 63 2 41 0 0 0 0 0 0 0 0 20 16 1 13 14 7 80 5 61 13 14 1 7 89 0 0 0 0 0 0 0 29 1 9 21 15 25 22 45 5 3 1 12 68 15 42 0 0 0 0 0 0 35 34 1 63 15 25 22 51 5 3 1 12 19 26 11 24 0 0 0 0 0 20 16 17 1 7 89 5 64 93 1 7 80 0 0 0 0 0 0 0 0 0 36 1 18 15 25 79 54 6 3 1 8 58 2 45 0 0 0 0 0 0 0 3 55 91 4 73 4 71 6 10 38 0 0 0 0 0 0 0 0 0 0 0 29 1 57 2 53 6 3 105 103 9 19 2 46 0 0 0 0 0 0 0 0 10 82 1 39 16 17 13 5 10 85 1 20 13 17 0 0 0 0 0 0 0 11 30 1 12 63 2 49 5 3 1 9 68 2 56 0 0 0 0 0 0 0 3 180 2 33 2 51 114 0 0 0 0 0 0 0 0 0 0 0 0 0 0 11 30 1 67 2 41 5 3 1 12 63 2 52 0 0 0 0 0 0 0 0 11 30 1 59 26 98 5 3 1 8 137 2 43 0 0 0 0 0 0 0 0 40 16 17 1 13 10 82 5 64 13 140 1 84 0 0 0 0 0 0 0 0 4 32 31 1 12 63 37 11 24 6 3 1 12 92 2 46 0 0 0 0 0 3 204 123 2 62 0 0 0 0 0 0 0 0 0

### Training the simple model on the training data set

Using the same model as above on the training data split to see how it performs on the training split of the data.

In [ ]:
### Trying the simple_model on the training split of the dataset

def simple_model_split(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Building the layers
    
    # Input layer
    input_layer = Input(shape=input_shape[1:])
    
    # GRU Layer
    x = GRU(256, return_sequences=True)(input_layer)
    
    # Fully-connected Layer
    x = Dense(french_vocab_size*4, activation='relu')(x)
    
    # Output Layer
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    # Creating model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compiling the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr = 0.01),
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length_train)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Training the model on the training set
simple_rnn_model_split = simple_model_split(
    tmp_x.shape,
    max_french_sequence_length_train,
    english_vocab_size_train + 1,
    french_vocab_size_train + 1)

simple_rnn_model_split.fit(tmp_x, preproc_french_sentences_train, batch_size=200, epochs=10, validation_split=0.2)

# Printing prediction(s)
print(logits_to_text(simple_rnn_model_split.predict(tmp_x[:1])[0], french_tokenizer_train))

### Testing the simple model on the test data set

Checking how the model trained on the training set data performs on the unseen test data.

In [ ]:
### Evaluating the model on the test set ###

# Reshaping the test data 
test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Rehsaping the test label data to fit the model
preproc_french_sentences_test = preproc_french_sentences_test.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Evaluating the model on the unseen test data
simple_rnn_model_score = simple_rnn_model_split.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Simple model accuracy on unseen test dataset: {0:.2f}%".format(simple_rnn_model_score[1]*100))

### Model 2: Embedding (Another Attempt after hours of failing the first model haha)
![RNN](images/embedding.png)
Earlier work done has turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [ ]:
### Embedding model trained on entire dataset ###

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: 
    
    # RNN model with embedding layer
    input_layer = Input(shape=input_shape[1:])
    embedding_layer = Embedding(512, english_vocab_size)(input_layer)
    x = GRU(512, return_sequences=True)(embedding_layer)
    x = TimeDistributed(Dense(french_vocab_size*4, activation='relu'))(x)
    output_layer = Dense(french_vocab_size, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compiling the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Testing the model
tests.test_embed_model(embed_model)

# TODO: Padding the input 
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# TODO: Training the neural network on the training data
embedding_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)
embedding_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Printing prediction(s)
print(logits_to_text(embedding_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

### Training the embedding model on the training data set

Using the same model as above on the training data split to see how it performs on the training split of the data.

In [ ]:
### Training the embedding model on the training dataset ###

def embed_model_split(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: 
    
    # RNN model with embedding layer
    input_layer = Input(shape=input_shape[1:])
    embedding_layer = Embedding(512, english_vocab_size)(input_layer)
    x = GRU(512, return_sequences=True)(embedding_layer)
    x = TimeDistributed(Dense(french_vocab_size*4, activation='relu'))(x)
    output_layer = Dense(french_vocab_size, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compiling the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Padding the input 
tmp_x = pad(preproc_english_sentences_train, preproc_french_sentences_train.shape[1])

# Training the neural network on the training data
embedding_model_split_train = embed_model_split(tmp_x.shape,
                                  max_french_sequence_length_train,
                                  english_vocab_size_train + 1,
                                  french_vocab_size_train + 1)

embedding_model_split_train.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Printing prediction(s)
print(logits_to_text(embedding_model_split_train.predict(tmp_x[:1])[0], french_tokenizer_train))

### Testing the embedding model on the test data set

Checking how the model trained on the training set data performs on the unseen test data.

In [ ]:
### Evaluating the embedding model on the unseen test data ###

test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2]))

embedding_model_score = embedding_model_split_train.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Embedding model accuracy on unseen test data: {0:.2f}%".format(embedding_model_score[1]*100))

### Model 3: Bidirectional RNNs (Third attempted model)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Building and training a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: 
    
    # Building the bidirectional RNN model
    input_layer = Input(shape=input_shape[1:])
    x = Bidirectional(GRU(512, return_sequences=True, recurrent_dropout=0.2))(input_layer)
    x = Dense(french_vocab_size*2, activation='relu')(x)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compiling the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_bd_model(bd_model)

# Padding the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Training and Printing prediction(s)

# Training the model
bidirectional_model = bd_model(tmp_x.shape,
                               max_french_sequence_length,
                               english_vocab_size + 1,
                               french_vocab_size + 1)
# Fitting the model
bidirectional_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Printing out the predictions
print(logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], french_tokenizer))

### Training the bidirectional model on the training data set

Using the same model as above on the training data split to see how it performs.

In [ ]:
### Training the bidirectional model on the training data set ###

def bd_model_split(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Building and training a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # Building the bidirectional RNN model
    input_layer = Input(shape=input_shape[1:])
    x = Bidirectional(GRU(512, return_sequences=True, recurrent_dropout=0.2))(input_layer)
    x = Dense(french_vocab_size, activation='relu')(x)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compiling the model
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

# Padding the input
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length_train)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Training the model
bidirectional_model_train = bd_model_split(tmp_x.shape,
                               max_french_sequence_length_train,
                               english_vocab_size_train + 1,
                               french_vocab_size_train + 1)
# Fitting the model
bidirectional_model_train.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Printing out the predictions
print(logits_to_text(bidirectional_model_train.predict(tmp_x[:1])[0], french_tokenizer_train))

### Testing the bidirectional model on the test data set

Checking how the model trained on the training set data performs on the unseen test data.

In [ ]:
### Evaluating the bidirectional model on the unseen test data set ###

test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

bidirectional_model_score = bidirectional_model_train.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Bidirectional model accuracy on test data: {0:.2f}%".format(bidirectional_model_score[1]*100))

### Model 4: Prototype (1750)
Using everything from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Building and training a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: 
    
    # Final model 2 - simplified version of final model 1
    # Added Dense layer with relu activation after first GRU layer instead of after 2nd GRU layer
    
    input_layer = Input(shape=input_shape[1:])
    embedding_layer = Embedding(english_vocab_size*2, english_vocab_size)(input_layer)
    x = Bidirectional(GRU(256))(embedding_layer)
    x = Dense(256, activation='relu')(x)
    x = RepeatVector(output_sequence_length)(x)
    x = Bidirectional(GRU(256, return_sequences=True))(x)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    
    ### Final model 1 ###
    
    # Build RNN model with embedding, encoder-decoder and bidirectional layers
    #input_layer = Input(shape=input_shape[1:])
    #embedding_layer = Embedding(750, english_vocab_size)(input_layer)
    #encoder = GRU(750)(embedding_layer)
    #repeat_vector = RepeatVector(output_sequence_length)(encoder)
    #decoder = Bidirectional(GRU(750, return_sequences=True))(repeat_vector)
    #dense_layer = TimeDistributed(Dense(french_vocab_size*2, activation='relu'))(decoder)
    #output_layer = Dense(french_vocab_size, activation='softmax')(dense_layer)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compiling the model
    learning_rate = 0.005
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')

## Prediction (1750)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Training neural network using model_final
    
    # Reshaping the input
    x = pad(x, y.shape[1])
    
    # Defining final_model
    final_model = model_final(x.shape, 
                              y.shape[1],
                              len(x_tk.word_index) + 1,
                              len(y_tk.word_index) + 1)
    # Fitting the final model
    final_model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)
    
    model = final_model

    
    ## No need to edit anything BELOW THIS LINE if forking this project
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

### Training the final model on the training data set
Checking the same model as above on the training data split to see how it performs on the training split of the data.

In [ ]:
### Training final model on training dataset ###

# Padding the input
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length_train)

# Training the model
final_model_train = model_final(tmp_x.shape,
                                max_french_sequence_length_train,
                                english_vocab_size_train + 1,
                                french_vocab_size_train + 1)
# Fitting the model
final_model_train.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Printing out the predictions
print(logits_to_text(final_model_train.predict(tmp_x[:1])[0], french_tokenizer_train))

### Testing the final model on the test data set

Checking how the model trained on the training set data performs on the unseen test data.

In [ ]:
### Evaluating the final_model trained on the training set on the test data set ###

# Padding the test data input
test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2]))
preproc_french_sentences_test = preproc_french_sentences_test.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

print(test_x.shape)
print(preproc_french_sentences_train.shape)
print(preproc_french_sentences_test.shape)

# Evaluating the model
final_model_test_score = final_model_train.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Final model accuracy on test dataset: {0:.2f}%".format(final_model_test_score[1]*100))

### Model Performance on Test Data

Checking how each of the models performed on the test dataset. 

In [ ]:
print("Simple model accuracy on unseen test dataset: {0:.2f}%".format(simple_rnn_model_score[1]*100))
print("Embedding model accuracy on unseen test data: {0:.2f}%".format(embedding_model_score[1]*100))
print("Bidirectional model accuracy on test data: {0:.2f}%".format(bidirectional_model_score[1]*100))
print("Final model accuracy on test dataset: {0:.2f}%".format(final_model_test_score[1]*100))